In [1]:
import numpy as np
#import scipy.misc
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib inline

import math
import random

#from random import shuffle

In [2]:
X = [1,2,3,4,5]
random.shuffle(X)
X

[3, 2, 4, 5, 1]

In [3]:
numNetworks = 100
numNodesPerNetwork = 100
inputFeatures = 10
outputFeatures = 5

globalInnovationNumber = 0

assert inputFeatures < numNodesPerNetwork
assert outputFeatures < numNodesPerNetwork

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Node:
    """
    This represents a neural network node with an integer label nodeId, 
    and a special type, one of ["input", "output", "hidden"]
    """
    def __init__(self, nodeId, nodeType):
        self.nodeId = nodeId
        assert nodeType == "input" or nodeType == "output" or nodeType == "hidden"
        self.nodeType = nodeType
        
    def copy(self):
        return Node(self.nodeId, self.nodeType)    
        
    def __str__(self):
        return "Node: " + str(self.nodeId) + ", " + self.nodeType
    
        
class Edge:
    """
    This class represent a connection (axon) in a neuroevolution network with its data.
    nodeIn and nodeOut are the ids of the input and output nodes,
    enabled is a boolean representing its ability to propogate into future neural networks,
    innovId is the labeled innovation number which corresponds to the same edge in other NNs,
    and weight is the normal edge weight.
    """
    def __init__(self, nodeIn, nodeOut, enabled, innovId, weight):
        self.nodeInId = nodeIn
        self.nodeOutId = nodeOut
        self.enabled = enabled
        self.innovId = innovId
        self.weight = weight
        
    def copy(self):
        return Edge(self.nodeInId, self.nodeOutId, self.enabled, self.innovId, self.weight)
        
    def __str__(self):
        return "Edge: " + str(self.nodeInId) + " -> " + str(self.nodeOutId) + ", Enabled: " + \
            str(self.enabled) + ", InnovID: " + str(self.innovId) + ", Weight: " + str(self.weight)
    
class NeuroevolutionNetwork:
    
    # self.nodes is a dictionary indexed by (node id -> node object)
    # self.edges is indexed by (in node id -> edge object, from in node to out node)
    def __init__(self, numNodes, numInput, numOutput):
        assert numInput + numOutput < numNodes
        
        self.nodes = dict() 
        self.edges = dict()
        self.listEdges = []
        self.numNodesCounter = 0
        
        self.numInput = numInput
        self.numOutput = numOutput
        self.initNodes = numNodes
        
        for _ in range(numInput):
            self.addNode("input")
        for _ in range(numOutput):
            self.addNode("output")
        for _ in range(numNodes - numInput - numOutput):
            self.addNode("hidden")
        
    # Special copy method to handle it ourselves, so it's not just a shallow/deep clone
    def copy(self):
        copyNetwork = NeuroevolutionNetwork(self.initNodes, self.numInput, self.numOutput)
        
        for nodeId, node in self.nodes.items():
            # For every node id, copy its node to the new network
            copyNetwork.nodes[nodeId] = node.copy()
            
        for nodeId, edgeList in self.edges.items():
            # For every node in id, copy the list of outgoing edges to the new network
            copyEdgeArr = []
            for edge in edgeList:
                copyEdgeArr.append(edge.copy())
            copyNetwork.edges[nodeId] = copyEdgeArr  
            
        for pair in self.listEdges:
            copyNetwork.listEdges.append(tuple(pair))
            
        return copyNetwork
    
    def printAllEdges(self):
        for nodeInId, nodeArr in self.edges.items():
            print(nodeInId)
            for node in nodeArr:
                print(str(node))
                
    def numNodes(self):
        return len(self.nodes)
    
    def numEdges(self):
        return len(self.listEdges)
        
    def addNode(self, nodeType):
        nodeId = self.numNodesCounter
        self.nodes[nodeId] = Node(nodeId, nodeType)
        self.numNodesCounter += 1
        return nodeId
        
    def hasNode(self, nodeId):
        return nodeId in self.nodes    
        
    # Return true if a connection exists between nodes with these ids,
    # a connection from in -> out
    def isConnected(self, nodeInId, nodeOutId):    
        """if hasNode(nodeInId) and hasNode(nodeOutId):
            if nodeInId in self.edges:
                connectionsOut = self.edges[nodeInId]
                for nodeOut in connectionsOut:
                    if nodeOut.nodeId == nodeOutId:
                        return True
        return False"""
        return (nodeInId, nodeOutId) in self.listEdges
    
    # Return true if there is a valid connection from in -> out
    # Invariant to whether or not the nodes are connected or not
    def allowedConnection(self, nodeInId, nodeOutId):
        if self.hasNode(nodeInId) and self.hasNode(nodeOutId):
            inNode = self.nodes[nodeInId]
            outNode = self.nodes[nodeOutId]
            doubleInput = inNode.nodeType == "input" and outNode.nodeType == "output"
            doubleOutput = inNode.nodeType == "input" and outNode.nodeType == "output"
            return not doubleInput and not doubleOutput
        else:
            return False
        
    # Variant to whether nodes are connected. To be able to add a new connection, the nodes
    # must follow these conditions:
    """
    Nodes exist
    Nodes are not connected in either direction
    The end node is not an input
    The start node is not an output
    New: the new edge cannot create a cycle
    """
    def canAddConnection(self, nodeInId, nodeOutId):
        assert nodeInId != nodeOutId
        if not self.allowedConnection(nodeInId, nodeOutId):
            return False
        inToOutC = self.isConnected(nodeInId, nodeOutId)
        outToInC = self.isConnected(nodeOutId, nodeInId)
        endInput = self.nodes[nodeOutId].nodeType == "input"
        startOutput = self.nodes[nodeInId].nodeType == "output"
        longPathExists = self.findPathBetween(nodeOutId, nodeInId) # Check for possible cycles
        return not (inToOutC or outToInC or endInput or startOutput or longPathExists)
        
    # Select a random pair of nodes, that have no connection either way,
    # and are not both input and not both output,
    # and add a connection between them, from in -> out.
    def mutateAddConnection(self):
        assert len(self.nodes) >= 2
        
        # Generate a shuffled list of possible future connections
        availIds = [node.nodeId for _,node in self.nodes.items()]
        lenIds = len(availIds)
        availablePairs = [(availIds[i], availIds[j]) for i in range(lenIds) for j in range(lenIds)]
        random.shuffle(availablePairs)
        
        # Find two nodes that have no connection
        # Once found, add the one new connection, and return
        while True: 
            if len(availablePairs) == 0: # No more possible connections to check
                return
            pair = availablePairs.pop() 
            if pair[0] == pair[1]:
                continue
            # Select a random connection, that is neither a self connection (does not exist),
            # nor an existing connection in either direction.
            # inToOutC = self.isConnected(pair[0], pair[1])
            # outToInC = self.isConnected(pair[1], pair[0])
            if self.canAddConnection(pair[0], pair[1]):
                global globalInnovationNumber
                self.addEdgeWithInnovId(pair[0], pair[1], True, globalInnovationNumber, edgeWeight=0.5)
                globalInnovationNumber += 1
                return
            
    def findEdge(self, nodeInId, nodeOutId):
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            possibleEdge = self.edges[nodeInId][nodeOutIndex]
            if possibleEdge.nodeOutId == nodeOutId:
                return possibleEdge
        return None
        
    def removeEdge(self, nodeInId, nodeOutId):
        assert nodeInId in self.edges
        assert (nodeInId, nodeOutId) in self.listEdges
        
        self.listEdges.remove((nodeInId, nodeOutId))
        
        # Look for the correct node within the outgoing connections
        # i.e. find the connection in -> out
        for nodeOutIndex in range(len(self.edges[nodeInId])):
            nodeOut = self.edges[nodeInId][nodeOutIndex]
            if nodeOut.nodeOutId == nodeOutId:
                self.edges[nodeInId].pop(nodeOutIndex)
                return  
            
    # Simple tree traversal, to check for a path between in -> out, of any length
    def findPathBetween(self, nodeInId, nodeOutId):
        fringe = [nodeInId]
        marked = set()
        while len(fringe) > 0:
            v = fringe.pop(0)
            if v == nodeOutId: # This case is if nodeInId == nodeOutId
                return True
            marked.add(v)
            if v not in self.edges: # No edges starting from v
                continue
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if w == nodeOutId:
                    return True
                if w not in marked:
                    fringe.append(w)
        return False
            
    def addEdgeWithInnovId(self, nodeInId, nodeOutId, enabled, innovId, edgeWeight):
        assert self.canAddConnection(nodeInId, nodeOutId)
        
        newEdge = Edge(nodeInId, nodeOutId, enabled, innovId, edgeWeight)
        
        self.listEdges.append((nodeInId, nodeOutId))
        if nodeInId not in self.edges:
            self.edges[nodeInId] = []
        self.edges[nodeInId].append(newEdge)
            
    """
    This mutates the graph by finding a random existing edge,
    and then splitting into two edges with a new connection in the middle,
    as per Stanley & Miikkulainen.
    """    
    def mutateSplitConnection(self):
        # Find a random edge to mutate, assuming one exists
        assert len(self.edges) > 0
        randomEdgeIndex = int(len(self.listEdges) * random.random())
        oldEdge = tuple(self.listEdges[randomEdgeIndex])
        nodeInId, nodeOutId = oldEdge[0], oldEdge[1]
        
        # TEMP:
        # self.removeEdge(nodeInId, nodeOutId)
        # No, do not remove the edge. It must be preserved to play a part in later
        # NN "reproduction" and similarity measures
        
        oldEdgeObj = self.findEdge(nodeInId, nodeOutId)
        oldEdgeObj.enabled = False
        
        newMiddleNode = self.addNode("hidden")
        
        global globalInnovationNumber
        self.addEdgeWithInnovId(nodeInId, newMiddleNode, True, globalInnovationNumber, edgeWeight=0.5)
        globalInnovationNumber += 1
        self.addEdgeWithInnovId(newMiddleNode, nodeOutId, True, globalInnovationNumber, edgeWeight=0.5)
        globalInnovationNumber += 1
        
    """
    Get all of the directed edges of the NN, but in reverse i.e. incoming edges,
    also representing an in-degree count indexed by vertex.
    """
    def getReversedEdges(self):
        reversedEdges = dict()
        
        for edgeInId, edgeList in self.edges.items():
            for edge in edgeList:
                if edge.nodeOutId not in reversedEdges:
                    reversedEdges[edge.nodeOutId] = []
                reversedEdges[edge.nodeOutId].append(edge.nodeInId)
        
        return reversedEdges
    
    """
    The new forwarding algorithm works as follows, in a DAG:
    obtain the edges of the graph, reversed;
    determine the in-degree of each vertex, using above;
    use Kahn's algorithm -> remove vertices of in-degree zero,
    repeat until end or a valid topological sort has been found;
    update vertices in topological order
    
    Proof that Kahn's algorithm terminates in a valid topo sort <-> graph is a DAG
    is left as an exercise for the reader.
    """
    
    """
    Return a list of the nodes 
    that represent a valid topological sort,
    starting with input edges (guaranteed to be in-degree zero).
    """
    def graphTopoSort(self):
        reversedEdges = self.getReversedEdges()
        
        marked = dict()
        topoSort = []
        
        for nodeId,_ in self.nodes.items():
            marked[nodeId] = False
        
        while len(topoSort) < len(self.nodes):
            degZeroIncomingIds = []
            
            for nodeId,_ in self.nodes.items():
                # Boolean short circ. V
                # The second case can only come about when connections are discarded
                if (nodeId not in reversedEdges or len(reversedEdges[nodeId]) == 0) and not marked[nodeId]: 
                    # nodeId must have an in-degree zero
                    degZeroIncomingIds.append(nodeId)
                    marked[nodeId] = True
                    
                    # No outgoing edges, we're done
                    if nodeId not in self.edges:
                        continue
                    
                    # Discard all outgoing connections
                    outgoingEdges = self.edges[nodeId]
                    
                    for edge in outgoingEdges:
                        if edge.nodeOutId in reversedEdges:
                            reversedEdges[edge.nodeOutId].remove(nodeId)
            
            topoSort = topoSort + degZeroIncomingIds
        return topoSort
    
    """
    Calculate the entire NN output, starting from the input.
    Again, the topological ordering is used to guarantee the nodes are calculated correctly
    i.e. all incoming edges have valid values.
    """
    def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        # nodeData is the final result of the NN, indexed by nodeId
        # Initialize the input, which is ideally of shape (input nodes X 1)
        nodeData = dict()
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
        
        # Update the nodes in topo order, using the incoming edges to determine weights and values
        reversedEdges = self.getReversedEdges()
        topoSort = self.graphTopoSort()
        
        for nodeId in topoSort:
            nodeType = self.nodes[nodeId].nodeType
            
            # Input nodes must not be overridden. Their degree is technically zero
            if nodeType == 'input':
                continue
            
            # Find the list of incoming edges
            incomingEdges = []
            if nodeId in reversedEdges:
                incomingEdges = reversedEdges[nodeId]
            
            result = 0
            if len(incomingEdges) == 0:
                result = 0.5
            else:
                nodeSum = 0
                # Sum up the incoming values, and take the activation function of the sum,
                # while noting that only enabled edges count in the calculation.
                for inVertex in incomingEdges:
                    inEdge = self.findEdge(inVertex, nodeId)
                    if inEdge.enabled:  
                        value = nodeData[inVertex]
                        weight = inEdge.weight
                        nodeSum += value * weight
                result = sigmoid(nodeSum)
                
            nodeData[nodeId] = result
        
        return nodeData
    
    def score(self, currentNodeData, desiredOutput, lossFunction):
        predictedOutput = np.zeros((self.numOutput))
        for i in range(0, self.numOutput):
            predictedOutput[i] = currentNodeData[self.numInput + i]
        assert predictedOutput.shape == desiredOutput.shape
        loss = lossFunction(predictedOutput, desiredOutput)
        return predictedOutput, desiredOutput, loss
        
    """def forwardStep(self, inputData):
        inputData = np.array(inputData)
        assert inputData.shape[0] == self.numInput
        
        for i in range(len(inputData)):
            nodeData[i] = inputData[i]
            
        # Use Dijkstra's algorithm to update the NN as we go, 
        # using the input data as the starting fringe.
        
        # Ideally, the NN is a DAG, but this will work even if not.
        
        nodeData = dict()
        prev = dict()
        marked = dict()
        
        queue = [i for i in range(len(self.numInput))]
        
        for nodeId, _ in self.nodes.items():
            nodeData[nodeId] = -9999
            prev[nodeId] = -1
            marked[nodeId] = False
        
        while len(queue) > 0:
            v = queue.pop() # A node id
            marked[v] = True
            for edge in self.edges[v]:
                w = edge.nodeOutId
                if :
                    nodeData[]"""

In [4]:
# Unit tests for neuroevolution network

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)

# Nodes are initialized correctly
"""
for k,v in testNet.nodes.items():
    print(v)
"""
    
# Edges are initialized correctly
print("-----------------------------------")
print("Test #2")
print(testNet.edges, testNet.listEdges)

print("-----------------------------------")
print("Test #3")
for _ in range(5):
    testNet.mutateAddConnection()

testNet.printAllEdges()
    
print(testNet.listEdges)

print("-----------------------------------")
print("Test #4")

testEdge = testNet.listEdges[0]
testNet.removeEdge(testEdge[0], testEdge[1])

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #5")

testNet.mutateSplitConnection()

testNet.printAllEdges()

print(testNet.listEdges)

print("-----------------------------------")
print("Test #6")

testNetCopy = testNet.copy()
testNetCopy.mutateAddConnection()

print("Original network: ")
testNet.printAllEdges()
print("Copy network: ")
testNetCopy.printAllEdges()

-----------------------------------
Test #2
{} []
-----------------------------------
Test #3
67
Edge: 67 -> 45, Enabled: True, InnovID: 0, Weight: 0.5
65
Edge: 65 -> 96, Enabled: True, InnovID: 1, Weight: 0.5
51
Edge: 51 -> 85, Enabled: True, InnovID: 2, Weight: 0.5
72
Edge: 72 -> 54, Enabled: True, InnovID: 3, Weight: 0.5
77
Edge: 77 -> 75, Enabled: True, InnovID: 4, Weight: 0.5
[(67, 45), (65, 96), (51, 85), (72, 54), (77, 75)]
-----------------------------------
Test #4
67
65
Edge: 65 -> 96, Enabled: True, InnovID: 1, Weight: 0.5
51
Edge: 51 -> 85, Enabled: True, InnovID: 2, Weight: 0.5
72
Edge: 72 -> 54, Enabled: True, InnovID: 3, Weight: 0.5
77
Edge: 77 -> 75, Enabled: True, InnovID: 4, Weight: 0.5
[(65, 96), (51, 85), (72, 54), (77, 75)]
-----------------------------------
Test #5
67
65
Edge: 65 -> 96, Enabled: True, InnovID: 1, Weight: 0.5
51
Edge: 51 -> 85, Enabled: True, InnovID: 2, Weight: 0.5
72
Edge: 72 -> 54, Enabled: True, InnovID: 3, Weight: 0.5
77
Edge: 77 -> 75, Enabl

In [5]:
testNode = Node(85, 'hidden')
print(testNode.copy())

Node: 85, hidden


In [6]:
"""
Calculate the distance delta between two neuroevolution networks.
This heuristic is important in determining a normalized fitness score,
which rewards successful organisms but ensures that one species does not take over.

In Stanley, Miikkulainen, this is defined as

delta = c_1 * E / N + c_2 * D / N + c_3 * W
where E is the number of excess genes,
D is the number of disjoint genes, 
N is the normalization factor (number of genes total),
and W is the average weight distance between shared genes.

Ideally, the weights have constraint 0 <= c_1, c_2, c_3 <= 1, 
and should also be consistent.
"""
def neuroNetworkDiff(nn1, nn2, c1, c2, c3):
    # Calculate excess number of genes by count
    excess = abs(nn1.numEdges() - nn2.numEdges())
    
    # Disjoint genes just need to be counted,
    # shared genes must have their weight differences averaged
    disjoint = 0
    sharedGenesDiff = []
    
    numGenes = max(nn1.numEdges(), nn2.numEdges())
    
    firstEdgeInnovIds = set()
    weightsByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            weightsByInnovId[edge.innovId] = edge.weight
    
    for _,outEdges in nn2.edges.items():
        for secondEdge in outEdges:
            if secondEdge.innovId in firstEdgeInnovIds:
                firstEdgeWeight = weightsByInnovId[secondEdge.innovId]
                secondEdgeWeight = secondEdge.weight
                # secondEdge = nn2.edges[node.nodeId]
                sharedGenesDiff.append(abs(firstEdgeWeight - secondEdgeWeight))
            else:
                disjoint += 1

    avgDiffW = np.sum(sharedGenesDiff) / len(sharedGenesDiff)
    
    # print(excess, disjoint, numGenes, avgDiffW)
    
    return c1 * excess / numGenes + \
        c2 * disjoint / numGenes + \
        c3 * avgDiffW
        

def crossNeuralNetworks(nn1, nn2):
    assert nn1.numInput == nn2.numInput
    assert nn1.numOutput == nn2.numOutput
    newNumNodes = max(nn1.numNodes(), nn2.numNodes())
    
    crossedNet = NeuroevolutionNetwork(newNumNodes, nn1.numInput, nn1.numOutput)
    
    firstEdgeInnovIds = set()
    secondEdgeInnovIds = set()
    firstEdgesByInnovId = dict() # Guaranteed to be surjective, stores weights of the first NN
    secondEdgesByInnovId = dict()
    enabledByInnovId = dict()
    
    for _,outEdges in nn1.edges.items():
        for edge in outEdges:
            firstEdgeInnovIds.add(edge.innovId)
            firstEdgesByInnovId[edge.innovId] = edge
            enabledByInnovId[edge.innovId] = edge.enabled
    
    for _,outEdges in nn2.edges.items():
        for edge in outEdges:
            secondEdgeInnovIds.add(edge.innovId)
            secondEdgesByInnovId[edge.innovId] = edge
            if edge.innovId in enabledByInnovId:
                enabledByInnovId[edge.innovId] = enabledByInnovId[edge.innovId] and edge.enabled
            else:
                enabledByInnovId[edge.innovId] = edge.enabled
    
    """
    As per Stanley and Miikkulainen, merge the edges together by their innovation ids.
    Disjoint/excess innovations transfer fully, shared innovations must merge,
    weight by average, enabled by AND
    """
    
    sharedInnov = firstEdgeInnovIds.intersection(secondEdgeInnovIds)
    onlyFirstInnov = firstEdgeInnovIds.difference(secondEdgeInnovIds)
    onlySecondInnov = secondEdgeInnovIds.difference(firstEdgeInnovIds)
    
    #nodeIn, nodeOut, enabled, innovId, weight)
    for innovId in onlyFirstInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
        
    for innovId in onlySecondInnov:
        baseEdge = secondEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, baseEdge.weight)
            
    for innovId in sharedInnov:
        baseEdge = firstEdgesByInnovId[innovId]
        inId = baseEdge.nodeInId
        outId = baseEdge.nodeOutId
        
        firstWeight = firstEdgesByInnovId[innovId].weight
        secondWeight = secondEdgesByInnovId[innovId].weight
        avgWeight = 0.5 * (firstWeight + secondWeight)
        
        if crossedNet.canAddConnection(inId, outId):
            crossedNet.addEdgeWithInnovId(inId, outId, enabledByInnovId[innovId], innovId, avgWeight)
    
    return crossedNet
                

def squaredLoss(predicted, desired):
    return np.sum((predicted - desired) ** 2)    



In [7]:
print("-----------------------------------")
print("Test #7")

testNet = NeuroevolutionNetwork(numNodesPerNetwork, inputFeatures, outputFeatures)
for _ in range(10):
    testNet.mutateAddConnection()
    
testNetMutate = testNet.copy()

for _ in range(2):
    testNetMutate.mutateAddConnection()
for _ in range(5):
    testNetMutate.mutateSplitConnection()
    
testNet.printAllEdges()
testNetMutate.printAllEdges()

print(neuroNetworkDiff(testNet, testNetMutate, 0.5, 0.25, 0.2))

print("-----------------------------------")
print("Test #8")

testEdges = testNet.getReversedEdges()
for nodeId,listEdges in testEdges.items():
    print(nodeId)
    print(listEdges)

print("-----------------------------------")
print("Test #9")

testNet.printAllEdges()
topoSort = testNet.graphTopoSort()
print(topoSort)

-----------------------------------
Test #7
98
Edge: 98 -> 22, Enabled: True, InnovID: 8, Weight: 0.5
90
Edge: 90 -> 87, Enabled: True, InnovID: 9, Weight: 0.5
15
Edge: 15 -> 60, Enabled: True, InnovID: 10, Weight: 0.5
85
Edge: 85 -> 78, Enabled: True, InnovID: 11, Weight: 0.5
9
Edge: 9 -> 72, Enabled: True, InnovID: 12, Weight: 0.5
80
Edge: 80 -> 59, Enabled: True, InnovID: 13, Weight: 0.5
45
Edge: 45 -> 88, Enabled: True, InnovID: 14, Weight: 0.5
20
Edge: 20 -> 85, Enabled: True, InnovID: 15, Weight: 0.5
40
Edge: 40 -> 50, Enabled: True, InnovID: 16, Weight: 0.5
38
Edge: 38 -> 13, Enabled: True, InnovID: 17, Weight: 0.5
98
Edge: 98 -> 22, Enabled: True, InnovID: 8, Weight: 0.5
90
Edge: 90 -> 87, Enabled: False, InnovID: 9, Weight: 0.5
Edge: 90 -> 100, Enabled: True, InnovID: 20, Weight: 0.5
Edge: 90 -> 103, Enabled: True, InnovID: 26, Weight: 0.5
15
Edge: 15 -> 60, Enabled: True, InnovID: 10, Weight: 0.5
85
Edge: 85 -> 78, Enabled: True, InnovID: 11, Weight: 0.5
9
Edge: 9 -> 72, Enab

In [8]:
print("-----------------------------------")
print("Test #10")

testNet = NeuroevolutionNetwork(15, 5, 5)
for _ in range(20):
    testNet.mutateAddConnection()
for _ in range(3):
    testNet.mutateSplitConnection()
for _ in range(20):
    testNet.mutateAddConnection()
    
testNet.printAllEdges()

-----------------------------------
Test #10
13
Edge: 13 -> 10, Enabled: True, InnovID: 30, Weight: 0.5
Edge: 13 -> 7, Enabled: True, InnovID: 31, Weight: 0.5
Edge: 13 -> 11, Enabled: True, InnovID: 35, Weight: 0.5
Edge: 13 -> 6, Enabled: True, InnovID: 43, Weight: 0.5
Edge: 13 -> 9, Enabled: True, InnovID: 62, Weight: 0.5
Edge: 13 -> 17, Enabled: True, InnovID: 69, Weight: 0.5
Edge: 13 -> 16, Enabled: True, InnovID: 74, Weight: 0.5
10
Edge: 10 -> 5, Enabled: True, InnovID: 32, Weight: 0.5
Edge: 10 -> 14, Enabled: False, InnovID: 34, Weight: 0.5
Edge: 10 -> 8, Enabled: True, InnovID: 39, Weight: 0.5
Edge: 10 -> 7, Enabled: True, InnovID: 42, Weight: 0.5
Edge: 10 -> 16, Enabled: False, InnovID: 52, Weight: 0.5
Edge: 10 -> 17, Enabled: True, InnovID: 54, Weight: 0.5
Edge: 10 -> 6, Enabled: True, InnovID: 70, Weight: 0.5
3
Edge: 3 -> 10, Enabled: True, InnovID: 33, Weight: 0.5
Edge: 3 -> 16, Enabled: True, InnovID: 65, Weight: 0.5
12
Edge: 12 -> 13, Enabled: True, InnovID: 36, Weight: 0.5

In [9]:
testNet.forwardStep([1.0, 0.0, 1.0, 0.0, 1.0])
# [16, 10, 4, 1, 97]
# -> [4,3.2,2,1,9.7]

{0: 1.0,
 1: 0.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 0.6603121272342829,
 6: 0.76494942053080828,
 7: 0.89999492590016217,
 8: 0.73429229210116098,
 9: 0.68977077952373211,
 10: 0.70691160597348324,
 11: 0.83723302425020052,
 12: 0.6923714766686917,
 13: 0.76086276310428125,
 14: 0.70364967350886232,
 15: 0.62245933120185459,
 16: 0.89224361475944658,
 17: 0.7744981982425293}

In [10]:
print("-----------------------------------")
print("Test #11")

# score(self, currentNodeData, desiredOutput, lossFunction):

inputData = np.array([1.0, 0.0, 1.0, 0.0, 1.0])
totalNodeData = testNet.forwardStep(inputData)
desired = np.array([0.0, 1.0, 0.0, 1.0, 0.0])

predicted, _, testLoss = testNet.score(totalNodeData, desired, squaredLoss)
print(predicted)
print(desired)
print(testLoss)

-----------------------------------
Test #11
[ 0.66031213  0.76494942  0.89999493  0.73429229  0.68977078]
[ 0.  1.  0.  1.  0.]
1.84763606125


In [11]:
print("-----------------------------------")
print("Test #12")

testNet = NeuroevolutionNetwork(15, 5, 5)

for _ in range(5):
    testNet.mutateAddConnection()
    
testNetMutate1 = testNet.copy()
testNetMutate2 = testNet.copy()

for _ in range(5):
    testNetMutate1.mutateAddConnection()
for _ in range(2):
    testNetMutate1.mutateSplitConnection()

for _ in range(5):
    testNetMutate2.mutateAddConnection()
for _ in range(2):
    testNetMutate2.mutateSplitConnection()
    
print("**************************")
testNet.printAllEdges()
print("**************************")
testNetMutate1.printAllEdges()
print("**************************")
testNetMutate2.printAllEdges()
print("**************************")

crossedNet = crossNeuralNetworks(testNetMutate1, testNetMutate2)
crossedNet.printAllEdges()

-----------------------------------
Test #12
**************************
11
Edge: 11 -> 10, Enabled: True, InnovID: 76, Weight: 0.5
14
Edge: 14 -> 9, Enabled: True, InnovID: 77, Weight: 0.5
2
Edge: 2 -> 14, Enabled: True, InnovID: 78, Weight: 0.5
0
Edge: 0 -> 14, Enabled: True, InnovID: 79, Weight: 0.5
3
Edge: 3 -> 11, Enabled: True, InnovID: 80, Weight: 0.5
**************************
11
Edge: 11 -> 10, Enabled: True, InnovID: 76, Weight: 0.5
Edge: 11 -> 8, Enabled: True, InnovID: 83, Weight: 0.5
14
Edge: 14 -> 9, Enabled: True, InnovID: 77, Weight: 0.5
Edge: 14 -> 8, Enabled: True, InnovID: 84, Weight: 0.5
2
Edge: 2 -> 14, Enabled: True, InnovID: 78, Weight: 0.5
0
Edge: 0 -> 14, Enabled: False, InnovID: 79, Weight: 0.5
Edge: 0 -> 11, Enabled: True, InnovID: 81, Weight: 0.5
Edge: 0 -> 16, Enabled: True, InnovID: 88, Weight: 0.5
3
Edge: 3 -> 11, Enabled: True, InnovID: 80, Weight: 0.5
Edge: 3 -> 13, Enabled: False, InnovID: 82, Weight: 0.5
Edge: 3 -> 10, Enabled: True, InnovID: 85, Weigh

In [45]:
"""
Initialize a world class, which contains multiple neural networks and functions to calculate on them 
and evolve them towards a goal.
"""

numNetworks = 100
numTotalNodes = 15
numInput = 5
numOutput = 5

baseMutations = 0
initialMutations = 10
iterAddMutations = 3
iterSplitMutations = 1

similarityThreshold = 0.25 # A distance less than this number implies two NNs are the same "species"
similarityPenalty = 0.1 # In Stanley, Miikkulainen, this number is 1, representing dividing by raw count

class World:
    
    def __init__(self, numNetworks, numTotalNodes, numInput, numOutput):
        self.networks = []
        
        baseNet = NeuroevolutionNetwork(numTotalNodes, numInput, numOutput)
        for _ in range(baseMutations):
            baseNet.mutateAddConnection()
            
        for _ in range(numNetworks - 1):
            baseNetMutate = baseNet.copy()
            for _ in range(initialMutations):
                baseNetMutate.mutateAddConnection()
            self.networks.append(baseNetMutate)
        
    def calcDiffNetMatrix(self):
        # Recommended values: 
        c1 = 0.5
        c2 = 0.25
        c3 = 0.2
        
        n = len(self.networks)
        results = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i < j: # Only calculate half of the table since diff(i,j) == diff(j,i)
                    results[i,j] = neuroNetworkDiff(self.networks[i], self.networks[j], c1, c2, c3)
        return results
        
    """
    Run a forward step through every neural network,
    and then rank all neural networks by an adjusted fitness score, defined as
    
    adj_score_i = raw_score_i / sum_j sh(delta(i,j))
    
    where delta(i,j) is the distance between NNs i and j,
    sh(x) is 1 if x > threshold, otherwise 0,
    and the sum is taken over all other NNs in the world.
    """
    def scoreNetworks(self, inputData, desiredData):
        diffMatrix = self.calcDiffNetMatrix()
        
        nnLoss = np.zeros((len(self.networks)))
        for netId in range(len(self.networks)):
            # Compute raw_score_i, using training data and a loss function
            net = self.networks[netId]
            totalNodeData = net.forwardStep(inputData)
            predicted, _, testLoss = testNet.score(totalNodeData, desiredData, squaredLoss)
            
            # Compute sum_j sh(delta(i,j))
            similarNNCount = 1
            for netId2 in range(len(self.networks)):
                if netId == netId2:
                    continue
                distanceNN = None
                if netId < netId2:
                    distanceNN = diffMatrix[netId, netId2]
                else:
                    distanceNN = diffMatrix[netId2, netId]
                if distanceNN < similarityThreshold:
                    similarNNCount += similarityPenalty
            
            nnLoss[netId] = testLoss # / similarNNCount 
            
        return nnLoss
    
    
    def scoreNetworksOnData(self, largeInputData, largeDesiredData):
        assert largeInputData.shape[0] == largeDesiredData.shape[0]
        data_n = largeInputData.shape[0]
        runningScores = np.zeros((len(self.networks)))
        
        for i in range(data_n):
            inputData = largeInputData[i]
            desiredData = largeDesiredData[i]
            indivScores = self.scoreNetworks(inputData, desiredData)
            indivScores /= data_n
            runningScores += indivScores
            
        return runningScores
    
    
    def evolveNetworks(self, largeInputData, largeDesiredData):
        testScores = self.scoreNetworksOnData(inputData, desiredData)
        testBestNNIndex = np.argsort(testScores)
        bestNetworkIds = testBestNNIndex[0:25]
        
        nextNetworks = []
        for i in range(len(bestNetworkIds)):
            for _ in range(4):
                netCopy = self.networks[bestNetworkIds[i]].copy()
                for __ in range(iterAddMutations):
                    netCopy.mutateAddConnection()
                for __ in range(iterSplitMutations):
                    netCopy.mutateSplitConnection()
                nextNetworks.append(netCopy)
        self.networks = nextNetworks
        
        return testScores

def getSortedIndexFromScore(scoresArr):
    pass
    

In [46]:
print("-----------------------------------")
print("Test #13")

inputData = np.array([1.0, 0.0, 1.0, 0.0, 1.0])
desiredData = np.array([0.0, 1.0, 0.0, 1.0, 0.0])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
testScores = testWorld.scoreNetworks(inputData, desiredData)

testBestNNIndex = np.argsort(testScores)

print(testScores)

print(testBestNNIndex)

-----------------------------------
Test #13
[ 1.33266474  1.36628263  1.27846738  1.39918538  1.50921704  1.49868535
  1.29071404  1.26388579  1.33314296  1.33776143  1.35772185  1.3612978
  1.12725564  1.26930709  1.28533395  1.2741746   0.9857444   1.35803209
  1.24975358  1.16440345  1.38208484  1.50233273  1.5053235   1.35975157
  1.35548689  1.2674435   1.47034281  1.12617899  1.36825055  1.53876591
  1.25        1.46324575  1.54288964  1.25580148  1.18406339  1.43542675
  1.3408748   1.66577685  1.43929144  1.28923385  1.4654132   1.30048763
  1.36442031  1.28786343  1.49710101  1.19270884  1.21443696  1.41574999
  1.12675027  1.19168942  1.27684542  1.41822638  1.27984485  1.32329174
  1.26051157  1.34180419  1.49888831  1.3408748   1.40150045  1.42197519
  1.42856124  1.08345294  1.52579242  1.59363673  1.07055369  1.31604242
  1.19926061  1.37546642  1.29270053  1.28540124  1.17552181  1.39918538
  1.37214082  1.3366575   1.49974527  1.26404483  1.48598295  1.35000199
  1.505

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


In [47]:
print("-----------------------------------")
print("Test #14")

inputData = np.array([[1.0, 0.0, 1.0, 0.0, 1.0], [0.24, 0.27, 0.39, 0.56, 0.71]])
desiredData = np.array([[0.0, 1.0, 0.0, 1.0, 0.0], [0.76, 0.73, 0.61, 0.44, 0.29]])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
testScores = testWorld.scoreNetworksOnData(inputData, desiredData)

testBestNNIndex = np.argsort(testScores)

print(testScores)

print(testBestNNIndex)

-----------------------------------
Test #14


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


[ 0.85609127  0.70684148  0.74456672  0.88465847  0.62380061  0.77334003
  0.71335828  0.80568582  0.75932199  0.70355168  0.76402804  0.71804795
  0.75589608  0.77192758  0.81068437  0.75472465  0.60987692  0.77291818
  0.65578166  0.7715491   0.73180287  0.65792639  0.75001299  0.78348986
  0.70802859  0.74953136  0.83867317  0.78035073  0.6364208   0.63299027
  0.81243152  0.88845186  0.68809296  0.6356693   0.67940058  0.64469591
  0.76682977  0.81046491  0.87964431  0.70363426  0.87283299  0.67001646
  0.77820285  0.7735495   0.85668354  0.76907265  0.72730964  0.83450746
  0.77663426  0.80493256  0.73061632  0.67813381  0.80245183  0.77150513
  0.67721477  0.86324759  0.63688109  0.77997828  0.67825008  0.85634584
  0.84540468  0.64272926  0.6737352   0.67389707  0.73828567  0.81101087
  0.80913548  0.68638338  0.76034555  0.73945215  0.69234242  0.77789134
  0.86406366  0.75272655  0.68967781  0.6986379   0.69705629  0.68008833
  0.68637881  0.73038162  0.77575395  0.87364104  0

In [48]:
print("-----------------------------------")
print("Test #15")

inputData = np.array([[1.0, 0.0, 1.0, 0.0, 1.0], [0.24, 0.27, 0.39, 0.56, 0.71]])
desiredData = np.array([[0.0, 1.0, 0.0, 1.0, 0.0], [0.76, 0.73, 0.61, 0.44, 0.29]])

testWorld = World(numNetworks, numTotalNodes, numInput, numOutput)
scores = None
for _ in range(15):
    scores = testWorld.evolveNetworks(inputData, desiredData)

print(scores)

testEvolvedNet = testWorld.networks[0]
testEvolvedNet.printAllEdges()

-----------------------------------
Test #15


C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars


AssertionError: 